In [53]:
!pip install "flask[async]"

In [73]:
from flask import Flask, request, jsonify
from threading import Thread
import json

In [74]:
import os
from openai import OpenAI
import json
from vllm import SamplingParams

In [107]:
app = Flask(__name__)
received_data = []

In [76]:
BASE_URL = f"http://localhost:3000/v1"
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-moe-16b-chat"

In [77]:
client = OpenAI(api_key="NULL", base_url=BASE_URL, timeout=None, max_retries=0)

In [105]:
import re

def extract_json_from_markdown(text):
    """
    Extracts the first JSON object found inside a Markdown block (```json ... ```)
    or just inside any triple backticks.
    """
    # Remove Markdown code fencing if it exists
    code_block = re.search(r"```(?:json)?\s*([\s\S]+?)```", text)
    if code_block:
        return code_block.group(1).strip()
    
    # Fallback: Try to find a JSON object directly
    json_object = re.search(r"\{[\s\S]+\}", text)
    if json_object:
        return json_object.group(0).strip()

    raise ValueError("No JSON object found in model output.")


In [171]:
import json
import uuid
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

class MeetingSchedulerAgent:
    def __init__(self, client, model_path, token_folder="Keys/"):
        self.client = client
        self.model_path = model_path
        self.token_folder = token_folder

    def _call_model(self, prompt: str):
        response = self.client.chat.completions.create(
            model=self.model_path,
            temperature=0.0,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content.strip()

    def parse_email_for_meeting(self, email_text: str) -> dict:
        prompt = f"""
You are an assistant that helps in scheduling meetings.

Your task is to extract the following from the email:
1. From the input text, extract all unique email addresses and return them as a comma-separated list.
2. Meeting duration in minutes.
3. Time constraints (e.g., 'next week').
4. A 7 individual dates based on the time constraint (e.g., ["13-07-2025",..., "19-07-2025"]).
5. Subject of email , it need to show if its urgent, workshop , etc.
6. Email Content.

Rules:
- If any participant is just a name, append '@amd.com' to it.
- Return only a **valid JSON object** with the following fields:
  - participant_emails (string)
  - meeting_duration (integer)
  - time_constraints (string)
  - window (array of strings)
  - email_subject (string)
  - email_content (string)

Do not reason step-by-step. Do not explain. Do not use markdown. Return only JSON. Begin your output with {{
Email: {email_text}
"""
        raw = self._call_model(prompt)
        try:
            return json.loads(raw)
        except json.JSONDecodeError:
            print("❌ Invalid JSON from parse_email:\n", raw)
            raise

    def retrieve_calendar_events(self, user_email, start, end):
        token_path = self.token_folder + user_email.split("@")[0] + ".token"
        creds = Credentials.from_authorized_user_file(token_path)
        service = build("calendar", "v3", credentials=creds)
        events_result = service.events().list(
            calendarId='primary', timeMin=start, timeMax=end,
            singleEvents=True, orderBy='startTime'
        ).execute()

        events = []
        for event in events_result.get('items', []):
            attendees = []
            try:
                attendees = [a['email'] for a in event['attendees']]
            except:
                attendees = ["SELF"]

            events.append({
                "StartTime": event["start"],
                "EndTime": event["end"],
                "NumAttendees": len(set(attendees)),
                "Attendees": list(set(attendees)),
                "Summary": event.get("summary", "No Summary")
            })
        return events

    def format_calendar_result(self, result):
        lines = []
        for entry in result:
            for email, blocks in entry.items():
                lines.append(f"\n📧 Email: {email}")
                for idx, block in enumerate(blocks):
                    for event in block:
                        summary = event.get("Summary", "No Summary")
                        attendees = ", ".join(event.get("Attendees", []))
                        num = event.get("NumAttendees", "Unknown")
                        start = event["StartTime"].get("dateTime") or event["StartTime"].get("date")
                        end = event["EndTime"].get("dateTime") or event["EndTime"].get("date")
                        lines.append(f"  🧾 {summary}: {start} → {end}, Attendees: {attendees}, NumAttendees: {num}")
        return "\n".join(lines)

    def set_attendees_with_events(self, attendee_emails, start, end):
        self.Attendees = []
        for email in attendee_emails:
            events = self.retrieve_calendar_events(email, start, end)
            self.Attendees.append({
                "email": email,
                "events": events
            })

    def schedule_meeting(self, original_email: str,emails:list) -> dict:
        meeting_json = self.parse_email_for_meeting(original_email)

        window_dates = [datetime.strptime(d, "%d-%m-%Y") for d in meeting_json["window"]]
        min_date = min(window_dates)
        max_date = max(window_dates)
        start_dt = min_date.strftime("%Y-%m-%dT00:00:00+05:30")
        end_dt = (max_date + timedelta(days=1)).strftime("%Y-%m-%dT00:00:00+05:30")

        users = emails #[u.strip() for u in meeting_json["participant_emails"]]
        # Ensure emails is a list
        if isinstance(emails, str):
            users = [u.strip() for u in emails.split(",")]
        else:
            users = emails 
        print(users)
        calendar_result = [{user: [self.retrieve_calendar_events(user, start_dt, end_dt)]} for user in users]

        self.set_attendees_with_events(users, start_dt, end_dt)

        result_str = self.format_calendar_result(calendar_result)

        scheduling_prompt = f"""
You are a smart calendar assistant with access to:
- A list of attendees (name + email)
- Each attendee’s calendar events for the next 7 days

Your task is to schedule or reschedule meetings based on the email provided.

Prioritization rules:
1. Events with high‑priority subjects (e.g., 'Urgent', 'Workshop', 'Conference', 'Client meet', 'Meet Now', 'Server Down') must not be displaced.
2. Events with more attendees or high-priority tags increase the importance of that time slot.
3. Low‑priority events (e.g., tea breaks, casual/private events) may be rescheduled.

Important:
- Do NOT ask anyone to reschedule the meeting being scheduled (e.g., the one described in the email).
- Only ask someone to reschedule **their existing calendar event** if it **conflicts** with the new meeting **and**:
  1. It's low priority, and
  2. There are no other free slots within working hours (9 AM – 6 PM).

Constraints:
- Only schedule between 9:00 AM and 6:00 PM.
- When asked to schedule a meeting:
   a. Check if the requested day/time is available.
   b. If not, suggest another slot within 7 days.
   c. If low-priority events block, suggest rescheduling them.

Rescheduling behavior:
- If user wants to reschedule a meeting:
   • Check event priority.
   • If it's important or affects many people:
     - Ask why they want to change.
     - Explain what could happen.
     - Try to convince them politely to reschedule their event so you could schedule this event.
- If no slot is found, only then suggest minimal reschedules (with polite justification).

     
Email: {original_email}
Attendee Events:
{result_str}

Note:
- If an attendee’s block contains the line "❌ No events found", it means they have no calendar events.
- In such cases, do not include any reschedule messages for that attendee.


Output format:
Return a valid JSON with:
- "EventStart": ISO datetime
- "EventEnd": ISO datetime
- "Duration_mins": integer
- "participants": comma-separated list of emails
- "reschedule_messages": array of objects:
  {{
    "to": string,
    "original_event": string,
    "original_time": string,
    "message": string
  }}

"""

        raw_output = self._call_model(scheduling_prompt)

        try:
            #cleaned_output = extract_json_from_markdown(raw_output)
            parsed_output = json.loads(raw_output)
        except (json.JSONDecodeError, ValueError) as e:
            print("❌ Invalid JSON from schedule_meeting:\n", raw_output)
            raise e

        # Format the final output as per desired structure
        final_result = {
            "Request_id": str(uuid.uuid4()),
            "Datetime": datetime.now().strftime("%d-%m-%YT%H:%M:%S"),
            "Location": "",
            "From": "",
            "Attendees": self.Attendees,
            "Subject": meeting_json.get("email_subject", ""),
            "EmailContent": meeting_json.get("email_content", original_email),
            "EventStart": parsed_output.get("EventStart", ""),
            "EventEnd": parsed_output.get("EventEnd", ""),
            "Duration_mins": str(parsed_output.get("Duration_mins", "")),
            "MetaData": {
                "reschedule_messages": parsed_output.get("reschedule_messages", [])
            }
        }

        return final_result


In [178]:
from openai import OpenAI

BASE_URL = "http://localhost:3000/v1"
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-moe-16b-chat"

client = OpenAI(api_key="NULL", base_url=BASE_URL)
agent = MeetingSchedulerAgent(client, model_path=MODEL_PATH)



In [179]:
app = Flask(__name__)
received_data = []

In [180]:
def your_meeting_assistant(data,emails): 
    # Your Agentic AI Calls 
    data_output = agent.schedule_meeting(data,emails)
    print(json.dumps(data_output, indent=2))
    #print(data_output)
    return data_output

In [181]:
import time

@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    
    attendees = ", ".join([att["email"] for att in data.get("Attendees", [])])
    subject = data.get("Subject", "")
    email_content = data.get("EmailContent", "")
    
    req_prompt = f"Subject: {subject}\nAttendees: {attendees}\nEmail Content: {email_content}"
    print(f"\nConstructed Prompt:\n{req_prompt}")
    print(attendees)
    print("inside")
    start = time.perf_counter()
    new_data =  your_meeting_assistant(req_prompt,attendees)  # Must be async
    end = time.perf_counter()
    # Filter out messages where original_event == email_subject
    filtered_reschedules = [
        msg for msg in new_data.get("MetaData", {}).get("reschedule_messages", [])
        if msg.get("original_event") != subject
    ]
    
    # Update the original dict
    new_data["MetaData"]["reschedule_messages"] = filtered_reschedules
     # Safely copy keys if they exist
    for key in ["Request_id", "Datetime", "Location", "From"]:
        if key in data:
            new_data[key] = data[key]

    duration = end - start
    print(f"\nTime taken to generate new_data: {duration:.2f} seconds")

    received_data.append(data)
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [ ]:
run_flask() 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://134.199.207.21:5000
Press CTRL+C to quit



Constructed Prompt:
Subject: Agentic AI Project Status Update
Attendees: usertwo.amd@gmail.com, userthree.amd@gmail.com
Email Content: Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.
usertwo.amd@gmail.com, userthree.amd@gmail.com
inside
['usertwo.amd@gmail.com', 'userthree.amd@gmail.com']


14.139.128.62 - - [20/Jul/2025 08:04:04] "POST /receive HTTP/1.1" 200 -


{
  "Request_id": "b26b2311-1470-4d9f-872e-53230668f735",
  "Datetime": "20-07-2025T08:04:04",
  "Location": "",
  "From": "",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": {
            "dateTime": "2025-07-15T18:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "EndTime": {
            "dateTime": "2025-07-16T09:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
        },
        {
          "StartTime": {
            "dateTime": "2025-07-16T18:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "EndTime": {
            "dateTime": "2025-07-17T09:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
       

...

{
  "Request_id": "4bcfaa9d-a078-4cb5-b924-9b8765540be8",
  "Datetime": "20-07-2025T08:04:08",
  "Location": "",
  "From": "",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": []
    },
    {
      "email": "userthree.amd@gmail.com",
      "events": []
    }
  ],
  "Subject": "Client Validation - Urgent",
  "EmailContent": "Hi Team. We\u00e2\u20ac\u2122ve just received quick feedback from the client indicating that the instructions we provided aren\u00e2\u20ac\u2122t working on their end. Let\u00e2\u20ac\u2122s prioritize resolving this promptly. Let\u00e2\u20ac\u2122s meet Monday at 9:00 AM to discuss and resolve this issue.",
  "EventStart": "2022-11-21T09:00:00.000Z",
  "EventEnd": "2022-11-21T09:30:00.000Z",
  "Duration_mins": "30",
  "MetaData": {
    "reschedule_messages": [
      {
        "to": "usertwo.amd@gmail.com",
        "original_event": "Client Validation - Urgent",
        "original_time": "Monday at 9:00 AM",
        "message": "Hi, I not

14.139.128.62 - - [20/Jul/2025 08:04:14] "POST /receive HTTP/1.1" 200 -


{
  "Request_id": "2b8b1da4-cb6b-45d8-bf7f-74b62d104d31",
  "Datetime": "20-07-2025T08:04:14",
  "Location": "",
  "From": "",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": {
            "dateTime": "2025-07-12T18:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "EndTime": {
            "dateTime": "2025-07-13T09:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
        },
        {
          "StartTime": {
            "date": "2025-07-13"
          },
          "EndTime": {
            "date": "2025-07-14"
          },
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Weekend"
        },
        {
          "StartTime": {
            "dateTime": "2025-07-13T18:00:00+05:30",
            "timeZone": "Asi

14.139.128.62 - - [20/Jul/2025 08:04:20] "POST /receive HTTP/1.1" 200 -


{
  "Request_id": "3ebcb201-f866-403b-8ee3-58ce7dbc8554",
  "Datetime": "20-07-2025T08:04:20",
  "Location": "",
  "From": "",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": {
            "dateTime": "2025-07-15T18:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "EndTime": {
            "dateTime": "2025-07-16T09:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
        },
        {
          "StartTime": {
            "dateTime": "2025-07-16T18:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "EndTime": {
            "dateTime": "2025-07-17T09:00:00+05:30",
            "timeZone": "Asia/Kolkata"
          },
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
       

In [36]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [39]:
!pip install psutil

In [41]:
import os
import psutil

def find_and_kill(port):
    for conn in psutil.net_connections(kind='inet'):
        la = conn.laddr
        if la and la.port == port:
            pid = conn.pid
            if pid:
                proc = psutil.Process(pid)
                print(f"Killing PID {pid} ({proc.name()}) listening on port {port}")
                os.kill(pid, 9)
                return
    print(f"No process found listening on port {port}")